In [1]:
'''
Sources:
https://forums.ni.com/t5/LabVIEW/regex-find-number-at-end-of-string/td-p/2510826?profile.language=en
https://numpy.org/devdocs/user/basics.types.html
https://www.geeksforgeeks.org/numpy-floor-python/
https://stackoverflow.com/questions/12201577/how-can-i-convert-an-rgb-image-into-grayscale-in-python
https://www.tutorialspoint.com/python/python_multithreading.htm
'''

'\nSources:\nhttps://forums.ni.com/t5/LabVIEW/regex-find-number-at-end-of-string/td-p/2510826?profile.language=en\nhttps://numpy.org/devdocs/user/basics.types.html\nhttps://www.geeksforgeeks.org/numpy-floor-python/\nhttps://stackoverflow.com/questions/12201577/how-can-i-convert-an-rgb-image-into-grayscale-in-python\nhttps://www.tutorialspoint.com/python/python_multithreading.htm\n'

In [2]:
import time
import threading
import concurrent.futures

from watchdog.observers import Observer
from watchdog.events import PatternMatchingEventHandler
from watchdog.events import RegexMatchingEventHandler

import cv2

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from skimage import data, img_as_float
from skimage import exposure

from sklearn.cluster import KMeans

from skimage.future import graph
import skimage.data as data
import skimage.segmentation as seg
import skimage.filters as filters
import skimage.draw as draw
import skimage.color as color
from skimage import img_as_float
from skimage import exposure
from skimage.color import rgb2gray
from skimage import io
import skimage.restoration as restore
from skimage.measure import compare_ssim as ssim
from skimage import img_as_ubyte

from sklearn.metrics import mean_squared_error
from math import log10, sqrt

import os
from os import walk
from os import listdir

from PIL import Image, ImageFilter, ImageFile

import shutil

from DirectoryGenerator import DirectoryGenerator

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import multiprocessing as mp
%matplotlib inline

In [3]:
def estimateAOIBoundingBoxes(imgDirectory, tempDirectory, finalDirectory, paramList, debugMode=False):
    def readImage(method, fileName):
        if(method == 'cv2'):
            return cv2.imread(fileName, cv2.IMREAD_GRAYSCALE)
        elif(method == 'skimage'):
            img = io.imread(fileName)
            return rgb2gray(img)
        elif(method == 'pil'):
            return Image.open(fileName).convert('LA')
        
    def saveImage(method, img, fileName):
        if(method == 'cv2'):
            cv2.imwrite(fileName, img)
        elif(method == 'skimage'):
            io.imsave(fileName, img_as_ubyte(np.uint8(np.floor(img))))
        elif(method == 'pil'):
            img.save(fileName)
    
    def sharpenImage(img, n=1):
        # Create our shapening kernel, it must equal to one eventually
        res = img
        for i in range(0, n):
            res = res.filter(ImageFilter.SHARPEN)
        return res
    
    def histogramEqualization(img):
        return exposure.equalize_hist(img) * 255
        
    def kMeansImageSegmentation(img, n_clusters=30, random_state=0):
        kmeans = KMeans(n_clusters=n_clusters, random_state=random_state).fit(img)
        pic2show = kmeans.cluster_centers_[kmeans.labels_]
        cluster_pic = pic2show.reshape(img.shape[0], img.shape[1])
        return cluster_pic
    
    def bilateralFilter(img, d=9, sigmaColor=500, sigmaSpace=1000):
        return cv2.bilateralFilter(img, d=d, sigmaColor=sigmaColor, sigmaSpace=sigmaSpace)
    
    def slic(img, compactness=2000, n_segments=8000):
        # Perform SLIC
        labels = seg.slic(img, compactness=compactness, n_segments=n_segments)
        return color.label2rgb(labels, img, kind='avg')

    img = readImage('pil', imgDirectory)
    img_sharp = sharpenImage(img, n=paramList['imgSharpening']['n'])
    saveImage('pil', img_sharp, tempDirectory + '_sharp.png' if debugMode else tempDirectory)
    
    img_sharp = readImage('cv2', tempDirectory + '_sharp.png' if debugMode else tempDirectory)
    img_hist_eq = histogramEqualization(img_sharp)
    saveImage('cv2', img_hist_eq, tempDirectory + '_histeq.png' if debugMode else tempDirectory)
    
    img_hist_eq = readImage('skimage', tempDirectory + '_histeq.png' if debugMode else tempDirectory)
    img_kmeans = kMeansImageSegmentation(img_hist_eq / 255, n_clusters=paramList['kmeans_1']['n_clusters'], random_state=paramList['kmeans_1']['random_state'])
    saveImage('skimage', img_kmeans * 255, tempDirectory + '_kmeans.png' if debugMode else tempDirectory)
    
    img_kmeans = readImage('cv2', tempDirectory + '_kmeans.png' if debugMode else tempDirectory)
    img_blur = bilateralFilter(img_kmeans, d=paramList['bilateralFilter']['d'], sigmaColor=paramList['bilateralFilter']['sigmaColor'], sigmaSpace=paramList['bilateralFilter']['sigmaSpace'])
    saveImage('cv2', img_blur, tempDirectory + '_blur.png' if debugMode else tempDirectory)
    
    img_blur = readImage('skimage', tempDirectory + '_blur.png' if debugMode else tempDirectory)
    img_slic = slic(img_blur, compactness=paramList['slic']['compactness'], n_segments=paramList['slic']['n_segments'])
    saveImage('skimage', img_slic, tempDirectory + '_slic.png' if debugMode else tempDirectory)
    
    img_slic = readImage('skimage', tempDirectory + '_slic.png' if debugMode else tempDirectory)
    img_slic_kmeans = kMeansImageSegmentation(img_slic / 255, n_clusters=paramList['kmeans_2']['n_clusters'], random_state=paramList['kmeans_2']['random_state'])
    saveImage('skimage', img_slic_kmeans * 255, finalDirectory)

In [4]:
def sceneSegmenterSSIM(videoFilePath, resultFolderName, sceneRefFolderName, threshold=0.70, recentFrameCompThreshold=0.99, framesPerSecond=1, totalFrames=0, useHeuristics=False):
    # Get the video file
    vidObj = cv2.VideoCapture(videoFilePath)
    # Track the number of frames read so far
    imageCount = 0
    # Track the number of scenes created so far
    sceneCount = 0
    # This array is used for storing the normalized representation of the first frame element of each scene
    # for the comparison process
    sceneRef = []
    recentFrame = None
    recentPredScene = None
    
    def convertToGrayscale(image):
        # Convert the current frame to grayscale
        return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
    def normalizeAdaptiveThreshold(imageGray):
        # Normalize the grayscale representation of the current frame using cv2.adaptiveThreshold
        # with cv2.ADAPTIVE_THRESH_GAUSSIAN_C and cv2.THRESH_BINARY
        return cv2.adaptiveThreshold(imageGray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)
    
    def predictSceneRecent(imageNorm):
        nonlocal sceneRef
        nonlocal imageCount
        nonlocal recentPredScene
        nonlocal recentFrame
        nonlocal recentFrameCompThreshold
        
        ssimVal = None
        isPart = False
        
        if(recentPredScene != None and recentFrame.all() != None):
            ssimVal = ssim(imageNorm, recentFrame)
            isPart = ssimVal >= recentFrameCompThreshold
            
            logFile = open('./' + resultFolderName + '/ssimLogs.txt', "a+")
            logInput = str(imageCount)
            logInput = logInput + " recentPredScene: " + str(recentPredScene) + " ssim: " + str(ssimVal) + " result: " + str(isPart) + "\n"
            logFile.write(logInput)
            logFile.close()
        
        return isPart
    
    def predictScene(imageNorm):
        nonlocal sceneRef
        nonlocal imageCount
        
        ssimResults = [ssim(imageNorm, scene) for scene in sceneRef]
        
        maxVal = None
        maxScene = None
        
        if(len(ssimResults) > 0):
            maxVal = max(ssimResults)
            maxScene = ssimResults.index(maxVal)
        
        logFile = open('./' + resultFolderName + '/ssimLogs.txt', "a+")
        logInput = str(imageCount)
        for i in range(0, len(ssimResults)):
            logInput = logInput + " " + str(i) + ": " + str(ssimResults[i])
        logInput = logInput + " maxVal: " + str(maxVal) + " predScene: " + str(maxScene) + "\n"
        logFile.write(logInput)
        logFile.close()
        
        return maxScene, maxVal
    
    def storeRecentScene(imageOrig, imageGray, imageNorm):
        nonlocal imageCount
        nonlocal sceneCount
        nonlocal resultFolderName
        nonlocal recentPredScene
        
        # Just save the grayscale representation of the current frame in the folder
        # pertaining to the predicted scene of that frame
        # as a PNG file with its file name being the current frame count
        cv2.imwrite('./' + resultFolderName + '/' + str(recentPredScene) + '/orig/frame%d.png' % imageCount, imageOrig)
        cv2.imwrite('./' + resultFolderName + '/' + str(recentPredScene) + '/gray/frame%d.png' % imageCount, imageGray)
        cv2.imwrite('./' + resultFolderName + '/' + str(recentPredScene) + '/norm/frame%d.png' % imageCount, imageNorm)
        
        return None
    
    def storeScene(imageOrig, imageGray, imageNorm, maxScene, maxVal):
        nonlocal threshold
        nonlocal imageCount
        nonlocal sceneCount
        nonlocal sceneRef
        nonlocal resultFolderName
        nonlocal recentPredScene
        nonlocal recentFrame
        
        # If the ssim value of the predicted scene of the current frame is less than the threshold value
        if(maxVal == None or maxVal < threshold):
            # Create a subfolder in "segmented_data_SSIM" named after the current no. of scenes
            # if it does not exists
            try:
                os.mkdir('./' + resultFolderName + '/' + str(sceneCount))
                os.mkdir('./' + resultFolderName + '/' + str(sceneCount) + '/orig')
                os.mkdir('./' + resultFolderName + '/' + str(sceneCount) + '/gray')
                os.mkdir('./' + resultFolderName + '/' + str(sceneCount) + '/norm')
            except(FileExistsError):
                print('./' + resultFolderName + '/' + str(sceneCount) + ' Exists')
            # Save the grayscale representation of the current frame in that folder
            # as a PNG file with its file name being the current frame count
            cv2.imwrite('./' + resultFolderName + '/' + str(sceneCount) + '/orig/frame%d.png' % imageCount, imageOrig)
            cv2.imwrite('./' + resultFolderName + '/' + str(sceneCount) + '/gray/frame%d.png' % imageCount, imageGray)
            cv2.imwrite('./' + resultFolderName + '/' + str(sceneCount) + '/norm/frame%d.png' % imageCount, imageNorm)
            # Append the normalized representation of the current frame to sceneRef
            sceneRef.append(imageNorm)
            # Increment the number of scenes
            recentPredScene = sceneCount
            sceneCount = sceneCount + 1
            try:
                os.mkdir('./' + sceneRefFolderName + '/' + str(sceneCount))
            except(FileExistsError):
                print('./' + sceneRefFolderName + '/' + str(sceneCount) + ' Exists')
            for i in range(0, len(sceneRef)):
                cv2.imwrite('./' + sceneRefFolderName + '/' + str(sceneCount) + '/frame%d_.png' % i, sceneRef[i])
        # Otherwise
        else:
            # Just save the grayscale representation of the current frame in the folder
            # pertaining to the predicted scene of that frame
            # as a PNG file with its file name being the current frame count
            cv2.imwrite('./' + resultFolderName + '/' + str(maxScene) + '/orig/frame%d.png' % imageCount, imageOrig)
            cv2.imwrite('./' + resultFolderName + '/' + str(maxScene) + '/gray/frame%d.png' % imageCount, imageGray)
            cv2.imwrite('./' + resultFolderName + '/' + str(maxScene) + '/norm/frame%d.png' % imageCount, imageNorm)
            recentPredScene = maxScene
        
        return None
    
    # While there are still frames to be read in the video file
    # Note: I added imageCount < 200 for the purpose of testing
    success = 1
    while(success and (imageCount < totalFrames if totalFrames > 0 else True)):
        # Obtain the next frame in the video file
        success, image = vidObj.read()
        
        if(imageCount % framesPerSecond == 0):
            imageGray = convertToGrayscale(image)
            imageNorm = normalizeAdaptiveThreshold(imageGray)
            if(useHeuristics):
                isPartOfRecentScene = predictSceneRecent(imageNorm)
                if(isPartOfRecentScene):
                    storeRecentScene(image, imageGray, imageNorm)
                else:
                    maxScene, maxVal = predictScene(imageNorm)
                    storeScene(image, imageGray, imageNorm, maxScene, maxVal)
                recentFrame = imageNorm
            else:
                maxScene, maxVal = predictScene(imageNorm)
                storeScene(image, imageGray, imageNorm, maxScene, maxVal)
        
        # Increment the number of frames read
        imageCount = imageCount + 1

In [5]:
# Sources:
# http://brunorocha.org/python/watching-a-directory-for-file-changes-with-python.html
# https://blog.magrathealabs.com/filesystem-events-monitoring-with-python-9f5329b651c3
# https://www.reddit.com/r/learnpython/comments/7kv2pc/using_watchdog_to_monitor_more_than_one_directory/
# https://stackoverflow.com/questions/17756011/how-to-match-only-particular-events-with-python-watchdog
# https://realpython.com/intro-to-python-threading/

def runVideoFrameSceneGrouping(videoFrameGroupingParams):
    folderObservers = []
    sceneObservers = []
    failedFrames = {}
    numberOfFrames = 0
    
    def AOIEstimationFailSafeThread(files):
        for file in files:
            imgDirectory = file
            tempDirectory = './' + imgDirectory.split('/')[0] + '/' + imgDirectory.split('/')[1] + '/temp/' + imgDirectory.split('/')[3]
            finalDirectory = './' + imgDirectory.split('/')[0] + '/' + imgDirectory.split('/')[1] + '/final/' + imgDirectory.split('/')[3]
            estimateAOIBoundingBoxes(imgDirectory, tempDirectory, finalDirectory, videoFrameGroupingParams['AOIEstimationParams'], videoFrameGroupingParams['AOIEstimationParams']['debugMode'])
            time.sleep(2)
    
    class SceneFolderHandler(PatternMatchingEventHandler):
        patterns = ['*.png']
        
        def setFolderName(self, folderName):
            self.folderName = folderName
    
        def incrementNumberOfFrames(self, lock):
            nonlocal numberOfFrames
            
            lock.acquire()
            numberOfFrames = numberOfFrames + 1
            lock.release()
        
        def process(self, event):
            nonlocal videoFrameGroupingParams
            nonlocal failedFrames
            """
            event.event_type 
                'modified' | 'created' | 'moved' | 'deleted'
            event.is_directory
                True | False
            event.src_path
                path/to/observed/file
            """
            # the file will be processed there
            try:
                if(videoFrameGroupingParams['AOIFailSafeDebugOn']):
                    failedFrames[self.folderName].append(str(event.src_path))
                else:
                    imgDirectory = str(event.src_path)
                    tempDirectory = './' + imgDirectory.split('/')[0] + '/' + imgDirectory.split('/')[1] + '/temp/' + imgDirectory.split('/')[3]
                    finalDirectory = './' + imgDirectory.split('/')[0] + '/' + imgDirectory.split('/')[1] + '/final/' + imgDirectory.split('/')[3]
                    estimateAOIBoundingBoxes(imgDirectory, tempDirectory, finalDirectory, videoFrameGroupingParams['AOIEstimationParams'], videoFrameGroupingParams['AOIEstimationParams']['debugMode'])
            except(Exception):
                failedFrames[self.folderName].append(str(event.src_path))
            finally:
                self.incrementNumberOfFrames(threading.Lock())
        
        def on_created(self, event):
            self.process(event)
            
    class MainFolderHandler(PatternMatchingEventHandler):
        def setFolderName(self, folderName):
            self.folderName = folderName

        def process(self, event):
            nonlocal sceneObservers
            nonlocal failedFrames
            """
            event.event_type 
                'modified' | 'created' | 'moved' | 'deleted'
            event.is_directory
                True | False
            event.src_path
                path/to/observed/file
            """
            # the file will be processed there
            if(event.is_directory):
                logFile = open('./' + str(self.folderName) + '.txt', "a+")
                logInput = str(event.src_path) + "/gray"
                logFile.write(logInput + "\n")
                logFile.close()
                
                try:
                    os.mkdir('./' + str(event.src_path) + '/temp')
                    os.mkdir('./' + str(event.src_path) + '/final')
                except(FileExistsError):
                    print('./' + str(event.src_path) + '/temp' + ' Exists')
                
                failedFrames[logInput] = []
            
                sceneObservers.append(Observer())
            
                handler = SceneFolderHandler()
                handler.setFolderName(logInput)
            
                sceneObservers[-1].schedule(handler, path=logInput)
                sceneObservers[-1].start()
            
        def on_created(self, event):
            self.process(event)
    
    folderObservers.append(Observer())
    
    try:
        os.mkdir('./' + videoFrameGroupingParams['resultFolderName'])
        os.mkdir('./' + videoFrameGroupingParams['resultFolderName'] + '/sceneRef')
    except(FileExistsError):
        print("Folder Exists!")
    
    logFile = open('./' + str(videoFrameGroupingParams['resultFolderName']) + '.txt', "a+")
    logFile.close()
    
    handler = MainFolderHandler()
    handler.setFolderName(videoFrameGroupingParams['resultFolderName'])
    
    folderObservers[-1].schedule(handler, path=videoFrameGroupingParams['resultFolderName'])
    folderObservers[-1].start()
    
    sceneSegmenterSSIM(videoFrameGroupingParams['video'], videoFrameGroupingParams['resultFolderName'], videoFrameGroupingParams['resultFolderName'] + '/sceneRef', videoFrameGroupingParams['threshold'], videoFrameGroupingParams['recentFrameCompThreshold'], videoFrameGroupingParams['framesPerSecond'], videoFrameGroupingParams['totalFrames'], videoFrameGroupingParams['useHeuristics'])
    
    try:
        while(numberOfFrames < videoFrameGroupingParams['totalFrames'] if videoFrameGroupingParams['totalFrames'] > 0 else True):
            time.sleep(1)
    except KeyboardInterrupt:
        print('Process Interrupted!')
    finally:
        with concurrent.futures.ThreadPoolExecutor(max_workers=len(failedFrames.values())) as executor:
            executor.map(AOIEstimationFailSafeThread, failedFrames.values())
        for folderObserver in folderObservers:
            folderObserver.unschedule_all()
            folderObserver.stop()
        for sceneObserver in sceneObservers:
            sceneObserver.unschedule_all()
            sceneObserver.stop()
            
    return failedFrames

In [6]:
class VideoFrameSceneGroupingMultipleVideos:
    def __init__(this):
        this.videoList = {}
    
    def addVideo(this, video, resultFolderName):
        this.videoList[resultFolderName] = {
            'video': video,
            'resultFolderName': resultFolderName,
            'AOIFailSafeDebugOn': False,
            'threshold': 0.70,
            'recentFrameCompThreshold': 0.99,
            'framesPerSecond': 1,
            'totalFrames': 0,
            'useHeuristics': False,
            'AOIEstimationParams': {
                'imgSharpening': {
                    'n': 1
                },
                'kmeans_1': {
                    'n_clusters': 30,
                    'random_state': 0
                },
                'bilateralFilter': {
                    'd': 9,
                    'sigmaColor': 500,
                    'sigmaSpace': 1000
                },
                'slic': {
                    'compactness': 2000,
                    'n_segments': 8000
                },
                'kmeans_2': {
                    'n_clusters': 30,
                    'random_state': 0
                },
                'debugMode': False
            }
        }
        
    def setAOIFailSafeDebugOn(this, resultFolderName, AOIFailSafeDebugOn):
        this.videoList[resultFolderName]['AOIFailSafeDebugOn'] = bool(AOIFailSafeDebugOn)
    
    def setThreshold(this, resultFolderName, threshold):
        this.videoList[resultFolderName]['threshold'] = float(threshold)
        
    def setRecentFrameCompThreshold(this, resultFolderName, recentFrameCompThreshold):
        this.videoList[resultFolderName]['recentFrameCompThreshold'] = float(recentFrameCompThreshold)
        
    def setFramesPerSecond(this, resultFolderName, framesPerSecond):
        this.videoList[resultFolderName]['framesPerSecond'] = int(framesPerSecond)
        
    def setTotalFrames(this, resultFolderName, totalFrames):
        this.videoList[resultFolderName]['totalFrames'] = int(totalFrames)
        
    def setUseHeuristics(this, resultFolderName, useHeuristics):
        this.videoList[resultFolderName]['useHeuristics'] = bool(useHeuristics)
        
    def setImgSharpeningN(this, resultFolderName, n):
        this.videoList[resultFolderName]['AOIEstimationParams']['imgSharpening']['n'] = int(n)
    
    def setKMeans1NClusters(this, resultFolderName, n_clusters):
        this.videoList[resultFolderName]['AOIEstimationParams']['kmeans_1']['n_clusters'] = int(n_clusters)
    
    def setKMeans1RandomState(this, resultFolderName, random_state):
        this.videoList[resultFolderName]['AOIEstimationParams']['kmeans_1']['random_state'] = int(random_state)
        
    def setBilateralFilterD(this, resultFolderName, d):
        this.videoList[resultFolderName]['AOIEstimationParams']['bilateralFilter']['d'] = int(d)
        
    def setBilateralFilterSigmaColor(this, resultFolderName, sigmaColor):
        this.videoList[resultFolderName]['AOIEstimationParams']['bilateralFilter']['sigmaColor'] = int(sigmaColor)
        
    def setBilateralFilterSigmaSpace(this, resultFolderName, sigmaSpace):
        this.videoList[resultFolderName]['AOIEstimationParams']['bilateralFilter']['sigmaSpace'] = int(sigmaSpace)
        
    def setSlicCompactness(this, resultFolderName, compactness):
        this.videoList[resultFolderName]['AOIEstimationParams']['slic']['compactness'] = int(compactness)
        
    def setSlicNSegments(this, resultFolderName, n_segments):
        this.videoList[resultFolderName]['AOIEstimationParams']['slic']['n_segments'] = int(n_segments)
        
    def setKMeans2NClusters(this, resultFolderName, n_clusters):
        this.videoList[resultFolderName]['AOIEstimationParams']['kmeans_2']['n_clusters'] = int(n_clusters)
    
    def setKMeans2RandomState(this, resultFolderName, random_state):
        this.videoList[resultFolderName]['AOIEstimationParams']['kmeans_2']['random_state'] = int(random_state)
        
    def setDebugMode(this, resultFolderName, debugMode):
        this.videoList[resultFolderName]['AOIEstimationParams']['debugMode'] = bool(debugMode)
        
    def runVideoFrameSceneGrouping(this):
        pool = mp.Pool(mp.cpu_count())
        print("Number of processors: ", mp.cpu_count())
        videoSeg = np.array(pool.map(runVideoFrameSceneGrouping, this.videoList.values()))
        pool.close()
        
        print(str(videoSeg))

In [7]:
class VideoFrameSceneGroupingParams:
    def __init__(this, video, resultFolderName):
        this.video = str(video)
        this.resultFolderName = str(resultFolderName)
        this.threshold = 0.70
        this.recentFrameCompThreshold = 0.99
        this.framesPerSecond = 1
        this.totalFrames = 850
        this.useHeuristics = False
    
    def setThreshold(this, threshold):
        this.threshold = float(threshold)
    
    def setRecentFrameCompThreshold(this, recentFrameCompThreshold):
        this.recentFrameCompThreshold = float(recentFrameCompThreshold)
        
    def setFramesPerSecond(this, framesPerSecond):
        this.framesPerSecond = int(framesPerSecond)
    
    def setTotalFrames(this, totalFrames):
        this.totalFrames = int(totalFrames)
    
    def setUseHeuristics(this, useHeuristics):
        this.useHeuristics = bool(useHeuristics)

In [8]:
def runVideoFrameSceneGroupingOnly(videoFrameGroupingParams):
    try:
        os.mkdir('./' + videoFrameGroupingParams.resultFolderName)
        os.mkdir('./' + videoFrameGroupingParams.resultFolderName + '/sceneRef')
    except(FileExistsError):
        print("Folder Exists!")
    
    start = time.time()
    sceneSegmenterSSIM(videoFrameGroupingParams.video, videoFrameGroupingParams.resultFolderName, videoFrameGroupingParams.resultFolderName + '/sceneRef', videoFrameGroupingParams.threshold, videoFrameGroupingParams.recentFrameCompThreshold, videoFrameGroupingParams.framesPerSecond, videoFrameGroupingParams.totalFrames, videoFrameGroupingParams.useHeuristics)
    end = time.time()
    logFile = open('./' + videoFrameGroupingParams.resultFolderName + '/ssimLogs.txt', "a+")
    logFile.write("Time Taken: " + str(end - start))
    logFile.close()

In [9]:
def testVideoFrameSceneGroupingTValue():
    testVideos = [VideoFrameSceneGroupingParams('ADDU-DI04 video_export_01-23-17-16.15.10.avi', 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_t5'),
                  VideoFrameSceneGroupingParams('ADDU-DI04 video_export_01-23-17-16.15.10.avi', 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_t9'),
                  VideoFrameSceneGroupingParams('ADDU-DI04 video_export_01-23-17-16.15.10.avi', 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_t7')]

    testVideos[0].setThreshold(0.5)
    testVideos[1].setThreshold(0.9)
    pool = mp.Pool(mp.cpu_count())
    print("Number of processors: ", mp.cpu_count())
    videoSeg = np.array(pool.map(runVideoFrameSceneGroupingOnly, testVideos))
    pool.close()

In [10]:
def testVideoFrameSceneGroupingHeuristicBenchmark():
    testVideos = [VideoFrameSceneGroupingParams('ADDU-DI04 video_export_01-23-17-16.15.10.avi', 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_nonheuristic_benchmark'),
                  VideoFrameSceneGroupingParams('ADDU-DI04 video_export_01-23-17-16.15.10.avi', 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_heuristic_99_benchmark')]
    testVideos[1].setUseHeuristics(True)

    pool = mp.Pool(mp.cpu_count())
    print("Number of processors: ", mp.cpu_count())
    videoSeg = np.array(pool.map(runVideoFrameSceneGroupingOnly, testVideos))
    pool.close()

In [11]:
def testVideoFrameSceneGroupingHeuristicBenchmarkTwo():
    testVideos = [VideoFrameSceneGroupingParams('ADDU-DI02 video_export_01-23-17-19.33.08.avi', 'ADDU-DI02 video_export_01-23-17-19.33.08.avi'.split(' ')[0] + '_nonheuristic_benchmark'),
                  VideoFrameSceneGroupingParams('ADDU-DI02 video_export_01-23-17-19.33.08.avi', 'ADDU-DI02 video_export_01-23-17-19.33.08.avi'.split(' ')[0] + '_heuristic_99_benchmark')]
    testVideos[1].setUseHeuristics(True)

    pool = mp.Pool(mp.cpu_count())
    print("Number of processors: ", mp.cpu_count())
    videoSeg = np.array(pool.map(runVideoFrameSceneGroupingOnly, testVideos))
    pool.close()

In [12]:
def testVideoFrameSceneGroupingCrossVideo():
    testVideos = [VideoFrameSceneGroupingParams('ADDU-DI02 video_export_01-23-17-19.33.08.avi', 'ADDU-DI02 video_export_01-23-17-19.33.08.avi'.split(' ')[0] + '_test'), 
                  VideoFrameSceneGroupingParams('ADDU-DI03 video_export_01-23-17-15.48.12.avi', 'ADDU-DI03 video_export_01-23-17-15.48.12.avi'.split(' ')[0] + '_test'), 
                  VideoFrameSceneGroupingParams('ADDU-DI04 video_export_01-23-17-16.15.10.avi', 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_test')]
    for i in range(0, len(testVideos)):
        testVideos[i].setUseHeuristics(True)

    pool = mp.Pool(mp.cpu_count())
    print("Number of processors: ", mp.cpu_count())
    videoSeg = np.array(pool.map(runVideoFrameSceneGroupingOnly, testVideos))
    pool.close()

In [13]:
def testVideoFrameSceneGroupingSingleVideo():
    test = VideoFrameSceneGroupingParams('ADDU-DI04 video_export_01-23-17-16.15.10.avi', 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest')
    test.setUseHeuristics(True)
    
    runVideoFrameSceneGroupingOnly(test)

In [14]:
def testAutomaticAOIBoundingBoxesEstimationAlgorithmDetailed():
    sample_data = ['./' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/0/gray/frame0.png', './' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/1/gray/frame18.png', './' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/2/gray/frame125.png', './' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/3/gray/frame139.png', './' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/6/gray/frame803.png']
    paramList = {
                    'imgSharpening': {
                        'n': 1
                    },
                    'kmeans_1': {
                        'n_clusters': 30,
                        'random_state': 0
                    },
                    'bilateralFilter': {
                        'd': 9,
                        'sigmaColor': 500,
                        'sigmaSpace': 1000
                    },
                    'slic': {
                        'compactness': 2000,
                        'n_segments': 8000
                    },
                    'kmeans_2': {
                        'n_clusters': 30,
                        'random_state': 0
                    }
                }
    
    for sample in sample_data:
        imgFileName = sample
        tempFileName = './' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/' + (imgFileName.split('/')[-1]).split('.')[0] + '_temp.png'
        finalFileName = './' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/' + (imgFileName.split('/')[-1]).split('.')[0] + '_aoi.png'
        estimateAOIBoundingBoxes(imgFileName, tempFileName, finalFileName, paramList, True)

In [15]:
def testAutomaticAOIBoundingBoxesEstimationAlgorithmBenchmark():
    #sample_data = ['./' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/0/gray/frame0.png', './' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/1/gray/frame18.png', './' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/2/gray/frame125.png', './' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/3/gray/frame139.png', './' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/6/gray/frame803.png']
    rootFolder = str(input())
    sample_data = [rootFolder + DirectoryGenerator().getDelimiter() + f for f in os.listdir(rootFolder) if f.endswith('.png')]
    paramList = {
                    'imgSharpening': {
                        'n': 1
                    },
                    'kmeans_1': {
                        'n_clusters': 30,
                        'random_state': 0
                    },
                    'bilateralFilter': {
                        'd': 9,
                        'sigmaColor': 500,
                        'sigmaSpace': 1000
                    },
                    'slic': {
                        'compactness': 2000,
                        'n_segments': 8000
                    },
                    'kmeans_2': {
                        'n_clusters': 30,
                        'random_state': 0
                    }
                }
    
    for sample in sample_data:
        imgFileName = sample
        #tempFileName = './' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/' + (imgFileName.split('/')[-1]).split('.')[0] + '_temp.png'
        #finalFileName = './' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/' + (imgFileName.split('/')[-1]).split('.')[0] + '_aoi.png'
        frameName = (imgFileName.split(DirectoryGenerator().getDelimiter())[-1]).split('.')[0]
        tempFileName = frameName + '_temp.png'
        finalFileName = frameName + '_aoi.png'
        
        start = time.time()
        estimateAOIBoundingBoxes(imgFileName, tempFileName, finalFileName, paramList, False)
        end = time.time()
        
        logFile = open('AutomaticAOIBoundingBoxesEstimationBenchmarks_Old.txt', "a+")
        logFile.write("Time Taken for " + frameName + ": " + str(end - start) + "\n")
        logFile.close()
    
    logFile = open('AutomaticAOIBoundingBoxesEstimationBenchmarks_Old.txt', "a+")
    logFile.write("\n")
    logFile.close()

In [16]:
# Sources:
# https://www.geeksforgeeks.org/python-peak-signal-to-noise-ratio-psnr/
# https://intellipaat.com/community/1269/is-there-a-library-function-for-root-mean-square-error-rmse-in-python

def testAutomaticAOIBoundingBoxesEstimationAlgorithmPSNR():
    def RMSE(imgOrig, imgSeg):
        return sqrt(mean_squared_error(imgOrig, imgSeg))
    
    def PSNR(imgOrig, imgSeg): 
        mse = mean_squared_error(imgOrig, imgSeg)
        if(mse == 0):
            return 100
        max_pixel = 255.0
        psnr = 20 * log10(max_pixel / sqrt(mse)) 
        return psnr 
    
    sample_data = ['./' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/0/gray/frame0.png', './' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/1/gray/frame18.png', './' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/2/gray/frame125.png', './' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/3/gray/frame139.png', './' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/6/gray/frame803.png']
    paramList = {
                    'imgSharpening': {
                        'n': 1
                    },
                    'kmeans_1': {
                        'n_clusters': 30,
                        'random_state': 0
                    },
                    'bilateralFilter': {
                        'd': 9,
                        'sigmaColor': 500,
                        'sigmaSpace': 1000
                    },
                    'slic': {
                        'compactness': 2000,
                        'n_segments': 8000
                    },
                    'kmeans_2': {
                        'n_clusters': 30,
                        'random_state': 0
                    }
                }
    
    for sample in sample_data:
        imgFileName = sample
        tempFileName = './' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/' + (imgFileName.split('/')[-1]).split('.')[0] + '_temp.png'
        finalFileName = './' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/' + (imgFileName.split('/')[-1]).split('.')[0] + '_aoi.png'
        
        estimateAOIBoundingBoxes(imgFileName, tempFileName, finalFileName, paramList, False)
        
        logFile = open('./' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/AutomaticAOIBoundingBoxesEstimationPSNR.txt', "a+")
        logFile.write("RMSE Value For " + (imgFileName.split('/')[-1]).split('.')[0] + ": " + str(RMSE(rgb2gray(io.imread(imgFileName)), rgb2gray(io.imread(finalFileName)))) + "\n")
        logFile.write("PSNR Value For " + (imgFileName.split('/')[-1]).split('.')[0] + ": " + str(PSNR(rgb2gray(io.imread(imgFileName)), rgb2gray(io.imread(finalFileName)))) + " dB\n")
        logFile.close()
    
    logFile = open('./' + 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_basictest/AutomaticAOIBoundingBoxesEstimationBenchmarks.txt', "a+")
    logFile.write("\n")
    logFile.close()

In [17]:
def testOverallThesis():
    videos = VideoFrameSceneGroupingMultipleVideos()

    videos.addVideo('ADDU-DI02 video_export_01-23-17-19.33.08.avi', 'ADDU-DI02 video_export_01-23-17-19.33.08.avi'.split(' ')[0] + '_final')
    videos.addVideo('ADDU-DI03 video_export_01-23-17-15.48.12.avi', 'ADDU-DI03 video_export_01-23-17-15.48.12.avi'.split(' ')[0] + '_final')
    videos.addVideo('ADDU-DI04 video_export_01-23-17-16.15.10.avi', 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_final')

    videos.setUseHeuristics('ADDU-DI02 video_export_01-23-17-19.33.08.avi'.split(' ')[0] + '_final', True)
    videos.setUseHeuristics('ADDU-DI03 video_export_01-23-17-15.48.12.avi'.split(' ')[0] + '_final', True)
    videos.setUseHeuristics('ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_final', True)

    videos.setTotalFrames('ADDU-DI02 video_export_01-23-17-19.33.08.avi'.split(' ')[0] + '_final', 850)
    videos.setTotalFrames('ADDU-DI03 video_export_01-23-17-15.48.12.avi'.split(' ')[0] + '_final', 850)
    videos.setTotalFrames('ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_final', 850)

    videos.runVideoFrameSceneGrouping()

In [18]:
def testOverallThesisAOIFailSafeDebug():
    videos = VideoFrameSceneGroupingMultipleVideos()

    videos.addVideo('ADDU-DI02 video_export_01-23-17-19.33.08.avi', 'ADDU-DI02 video_export_01-23-17-19.33.08.avi'.split(' ')[0] + '_final')
    videos.addVideo('ADDU-DI03 video_export_01-23-17-15.48.12.avi', 'ADDU-DI03 video_export_01-23-17-15.48.12.avi'.split(' ')[0] + '_final')
    videos.addVideo('ADDU-DI04 video_export_01-23-17-16.15.10.avi', 'ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_final')

    videos.setAOIFailSafeDebugOn('ADDU-DI02 video_export_01-23-17-19.33.08.avi'.split(' ')[0] + '_final', True)
    videos.setAOIFailSafeDebugOn('ADDU-DI03 video_export_01-23-17-15.48.12.avi'.split(' ')[0] + '_final', True)
    videos.setAOIFailSafeDebugOn('ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_final', True)

    videos.setTotalFrames('ADDU-DI02 video_export_01-23-17-19.33.08.avi'.split(' ')[0] + '_final', 50)
    videos.setTotalFrames('ADDU-DI03 video_export_01-23-17-15.48.12.avi'.split(' ')[0] + '_final', 50)
    videos.setTotalFrames('ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_final', 50)
    
    videos.setUseHeuristics('ADDU-DI02 video_export_01-23-17-19.33.08.avi'.split(' ')[0] + '_final', True)
    videos.setUseHeuristics('ADDU-DI03 video_export_01-23-17-15.48.12.avi'.split(' ')[0] + '_final', True)
    videos.setUseHeuristics('ADDU-DI04 video_export_01-23-17-16.15.10.avi'.split(' ')[0] + '_final', True)

    videos.runVideoFrameSceneGrouping()

In [19]:
#testVideoFrameSceneGroupingTValue()
#testVideoFrameSceneGroupingCrossVideo()
#testVideoFrameSceneGroupingHeuristicBenchmark()
#testVideoFrameSceneGroupingHeuristicBenchmarkTwo()
#testVideoFrameSceneGroupingSingleVideo()
#testAutomaticAOIBoundingBoxesEstimationAlgorithmDetailed()
testAutomaticAOIBoundingBoxesEstimationAlgorithmBenchmark()
#testAutomaticAOIBoundingBoxesEstimationAlgorithmPSNR()
#testOverallThesis()
#testOverallThesisAOIFailSafeDebug()

/Users/ezekieladrieldlagmay/Documents/Dynamic Individual Eye-Tracking/AutomaticAOIBoundingBoxes-V2/SamplePhotos


<ipython-input-3-651e12f0caac>:7: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  return rgb2gray(img)
<ipython-input-3-651e12f0caac>:7: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  return rgb2gray(img)
<ipython-input-3-651e12f0caac>:40: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
  labels = seg.slic(img, compactness=compactness, n_segments=n_segments)
<ipython-input-3-651e12f0caac>:41: FutureWarning: The new recommende

<ipython-input-3-651e12f0caac>:41: FutureWarning: The new recommended value for bg_label is 0. Until version 0.19, the default bg_label value is -1. From version 0.19, the bg_label default value will be 0. To avoid this warning, please explicitly set bg_label value.
  return color.label2rgb(labels, img, kind='avg')
<ipython-input-3-651e12f0caac>:7: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  return rgb2gray(img)
<ipython-input-3-651e12f0caac>:7: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  return rgb2gray(img)
<ipython-input-3-651e12f0caac>:40: Fu